# NYC Yellow Taxi Data Analysis

Анализ данных NYC Yellow Taxi 2025 года с использованием Spark Connect.

**Данные:** ~653 MB, 10 месяцев (январь-октябрь 2025), ~40 млн поездок  
**Источник:** [NYC TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

---

## Web UI

### Docker Compose
| Сервис | URL | Описание |
|--------|-----|----------|
| **Jupyter** | http://localhost:8888 | This notebook |
| **Spark UI** | http://localhost:4040 | Jobs, Stages, Storage, SQL |
| **Spark History** | http://localhost:18080 | История завершенных приложений |
| **MinIO Console** | http://localhost:9001 | S3 bucket browser (minioadmin/minioadmin) |

### Kubernetes
| Сервис | Команда / URL | Описание |
|--------|---------------|----------|
| **Jupyter** | http://localhost:30888 | NodePort |
| **Spark UI** | `kubectl port-forward -n spark svc/spark-connect 4040:4040` | Затем http://localhost:4040 |
| **Spark History** | `kubectl port-forward -n spark svc/spark-history-server 18080:18080` | Затем http://localhost:18080 |
| **MinIO Console** | `kubectl port-forward -n spark svc/minio 9001:9001` | Затем http://localhost:9001 |

---

In [ ]:
# Setup - прямое подключение к Spark Connect
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Подключение к Spark Connect серверу
spark = SparkSession.builder \
    .appName("NYCTaxiAnalysis") \
    .remote(os.environ.get('SPARK_REMOTE', 'sc://spark-connect:15002')) \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"\nSpark UI:")
print(f"  Docker Compose: http://localhost:4040")
print(f"  K8s: kubectl port-forward -n spark svc/spark-connect 4040:4040")

## 1. Загрузка данных из S3

In [ ]:
# Чтение всех parquet файлов
df = spark.read.parquet("s3a://raw-data/nyc-taxi/*.parquet")

# Кэширование для повторного использования
df.cache()

# Общее количество записей
total_count = df.count()
print(f"Всего записей: {total_count:,}")

In [ ]:
# Схема данных
df.printSchema()

In [ ]:
# Примеры данных
df.select(
    "tpep_pickup_datetime", 
    "tpep_dropoff_datetime",
    "passenger_count",
    "trip_distance", 
    "fare_amount", 
    "tip_amount", 
    "total_amount"
).show(10, truncate=False)

## 2. Базовая статистика

In [ ]:
# Общая статистика
stats = df.select(
    F.count("*").alias("total_trips"),
    F.round(F.avg("trip_distance"), 2).alias("avg_distance_miles"),
    F.round(F.avg("fare_amount"), 2).alias("avg_fare"),
    F.round(F.avg("tip_amount"), 2).alias("avg_tip"),
    F.round(F.avg("total_amount"), 2).alias("avg_total"),
    F.round(F.sum("total_amount") / 1_000_000, 2).alias("total_revenue_millions")
)

stats.show()

In [ ]:
# Статистика по числовым колонкам
df.select("trip_distance", "fare_amount", "tip_amount", "total_amount") \
    .summary("count", "mean", "stddev", "min", "25%", "50%", "75%", "max") \
    .show()

## 3. Анализ по месяцам

In [ ]:
# Агрегация по месяцам
monthly = df.withColumn("month", F.month("tpep_pickup_datetime")) \
    .groupBy("month") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.sum("total_amount") / 1_000_000, 2).alias("revenue_M"),
        F.round(F.avg("trip_distance"), 2).alias("avg_distance"),
        F.round(F.avg("fare_amount"), 2).alias("avg_fare"),
        F.round(F.avg("tip_amount"), 2).alias("avg_tip")
    ) \
    .orderBy("month")

monthly.show(12)

In [ ]:
# Визуализация
import matplotlib.pyplot as plt

monthly_pd = monthly.toPandas()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Поездки по месяцам
axes[0].bar(monthly_pd['month'], monthly_pd['trips'] / 1_000_000, color='steelblue')
axes[0].set_xlabel('Месяц')
axes[0].set_ylabel('Поездки (млн)')
axes[0].set_title('NYC Yellow Taxi: Поездки по месяцам (2025)')
axes[0].set_xticks(range(1, 11))

# Выручка по месяцам
axes[1].bar(monthly_pd['month'], monthly_pd['revenue_M'], color='green')
axes[1].set_xlabel('Месяц')
axes[1].set_ylabel('Выручка ($ млн)')
axes[1].set_title('NYC Yellow Taxi: Выручка по месяцам (2025)')
axes[1].set_xticks(range(1, 11))

plt.tight_layout()
plt.show()

## 4. Анализ по часам дня

In [ ]:
# Поездки по часам
hourly = df.withColumn("hour", F.hour("tpep_pickup_datetime")) \
    .groupBy("hour") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.avg("fare_amount"), 2).alias("avg_fare"),
        F.round(F.avg("trip_distance"), 2).alias("avg_distance")
    ) \
    .orderBy("hour")

hourly_pd = hourly.toPandas()

plt.figure(figsize=(14, 5))
plt.bar(hourly_pd['hour'], hourly_pd['trips'] / 1_000_000, color='coral')
plt.xlabel('Час дня')
plt.ylabel('Поездки (млн)')
plt.title('NYC Yellow Taxi: Распределение по часам дня')
plt.xticks(range(0, 24))
plt.axvspan(7, 9, alpha=0.3, color='red', label='Утренний час пик')
plt.axvspan(17, 19, alpha=0.3, color='red', label='Вечерний час пик')
plt.legend()
plt.show()

## 5. Анализ по дням недели

In [ ]:
# День недели (1=Sunday, 7=Saturday в Spark)
dow_names = {1: 'Вс', 2: 'Пн', 3: 'Вт', 4: 'Ср', 5: 'Чт', 6: 'Пт', 7: 'Сб'}

by_dow = df.withColumn("dow", F.dayofweek("tpep_pickup_datetime")) \
    .groupBy("dow") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.avg("total_amount"), 2).alias("avg_total"),
        F.round(F.avg("tip_amount"), 2).alias("avg_tip")
    ) \
    .orderBy("dow")

by_dow.show()

## 6. Типы оплаты

In [ ]:
# Типы оплаты
# 1=Credit card, 2=Cash, 3=No charge, 4=Dispute, 5=Unknown, 6=Voided

payment = df.groupBy("payment_type") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.sum("total_amount") / 1_000_000, 2).alias("revenue_M"),
        F.round(F.avg("tip_amount"), 2).alias("avg_tip"),
        F.round(F.avg("tip_amount") / F.avg("fare_amount") * 100, 1).alias("tip_pct")
    ) \
    .withColumn("payment_name", 
        F.when(F.col("payment_type") == 1, "Credit Card")
         .when(F.col("payment_type") == 2, "Cash")
         .when(F.col("payment_type") == 3, "No Charge")
         .when(F.col("payment_type") == 4, "Dispute")
         .otherwise("Other")
    ) \
    .orderBy(F.desc("trips"))

payment.select("payment_name", "trips", "revenue_M", "avg_tip", "tip_pct").show()

## 7. Топ локаций

In [ ]:
# Топ 10 точек посадки
top_pickups = df.groupBy("PULocationID") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.sum("total_amount"), 0).alias("total_revenue")
    ) \
    .orderBy(F.desc("trips")) \
    .limit(10)

print("Топ 10 точек посадки (PULocationID):")
top_pickups.show()

In [ ]:
# Топ маршрутов
top_routes = df.groupBy("PULocationID", "DOLocationID") \
    .agg(
        F.count("*").alias("trips"),
        F.round(F.avg("trip_distance"), 2).alias("avg_dist"),
        F.round(F.avg("fare_amount"), 2).alias("avg_fare"),
        F.round(F.avg("total_amount"), 2).alias("avg_total")
    ) \
    .orderBy(F.desc("trips")) \
    .limit(15)

print("Топ 15 маршрутов (PU -> DO):")
top_routes.show()

## 8. SQL аналитика

In [ ]:
# Регистрация временной таблицы
df.createOrReplaceTempView("taxi_trips")

In [ ]:
# Анализ по периодам дня
spark.sql("""
SELECT 
    CASE 
        WHEN hour(tpep_pickup_datetime) BETWEEN 6 AND 9 THEN '1. Утренний час пик'
        WHEN hour(tpep_pickup_datetime) BETWEEN 10 AND 15 THEN '2. День'
        WHEN hour(tpep_pickup_datetime) BETWEEN 16 AND 19 THEN '3. Вечерний час пик'
        WHEN hour(tpep_pickup_datetime) BETWEEN 20 AND 23 THEN '4. Вечер'
        ELSE '5. Ночь'
    END as time_period,
    COUNT(*) as trips,
    ROUND(AVG(trip_distance), 2) as avg_distance,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(tip_amount), 2) as avg_tip,
    ROUND(SUM(total_amount) / 1000000, 2) as revenue_M
FROM taxi_trips
WHERE fare_amount > 0
GROUP BY 1
ORDER BY 1
""").show()

In [ ]:
# Распределение по дистанции
spark.sql("""
SELECT 
    CASE 
        WHEN trip_distance < 1 THEN '< 1 mile'
        WHEN trip_distance < 3 THEN '1-3 miles'
        WHEN trip_distance < 5 THEN '3-5 miles'
        WHEN trip_distance < 10 THEN '5-10 miles'
        WHEN trip_distance < 20 THEN '10-20 miles'
        ELSE '20+ miles'
    END as distance_bucket,
    COUNT(*) as trips,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 1) as pct,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(tip_amount), 2) as avg_tip
FROM taxi_trips
WHERE trip_distance > 0 AND fare_amount > 0
GROUP BY 1
ORDER BY MIN(trip_distance)
""").show()

In [ ]:
# Поездки в/из аэропортов (Airport_fee > 0)
spark.sql("""
SELECT 
    CASE WHEN Airport_fee > 0 THEN 'Airport Trip' ELSE 'Regular Trip' END as trip_type,
    COUNT(*) as trips,
    ROUND(AVG(trip_distance), 2) as avg_distance,
    ROUND(AVG(fare_amount), 2) as avg_fare,
    ROUND(AVG(total_amount), 2) as avg_total,
    ROUND(AVG(tip_amount), 2) as avg_tip
FROM taxi_trips
GROUP BY 1
ORDER BY trips DESC
""").show()

## 9. Оконные функции

In [ ]:
# Кумулятивная выручка по месяцам
monthly_cumulative = df.withColumn("month", F.month("tpep_pickup_datetime")) \
    .groupBy("month") \
    .agg(F.round(F.sum("total_amount") / 1_000_000, 2).alias("revenue_M")) \
    .withColumn("cumulative_revenue_M", 
        F.sum("revenue_M").over(Window.orderBy("month"))
    ) \
    .withColumn("month_over_month_pct",
        F.round(
            (F.col("revenue_M") - F.lag("revenue_M").over(Window.orderBy("month"))) / 
            F.lag("revenue_M").over(Window.orderBy("month")) * 100, 1
        )
    ) \
    .orderBy("month")

monthly_cumulative.show()

In [ ]:
# Ранжирование локаций по месяцам
location_monthly = df.withColumn("month", F.month("tpep_pickup_datetime")) \
    .groupBy("month", "PULocationID") \
    .agg(F.count("*").alias("trips")) \
    .withColumn("rank", 
        F.rank().over(Window.partitionBy("month").orderBy(F.desc("trips")))
    ) \
    .filter(F.col("rank") <= 3) \
    .orderBy("month", "rank")

print("Топ-3 локации по месяцам:")
location_monthly.show(30)

## 10. pandas API on Spark

pandas API on Spark (бывший Koalas) позволяет использовать привычный pandas синтаксис для распределенных вычислений.

### ⚠️ Ограничение Spark Connect с TimestampNTZType

В Spark Connect есть известная проблема: `ps.read_parquet()` и `df.pandas_api()` вызывают `AssertionError` при наличии колонок типа `TimestampNTZType`.

**Пример ошибки:**
```
AssertionError: [InternalField(dtype=datetime64[us], struct_field=StructField('tpep_pickup_datetime', TimestampNTZType(), True))...]
```

**Проблемные колонки в NYC Taxi:** `tpep_pickup_datetime`, `tpep_dropoff_datetime`

### Workaround: исключение timestamp колонок

**Вариант 1: Выбор числовых колонок**
```python
# Читаем через Spark DataFrame (работает со всеми типами)
df = spark.read.parquet("s3a://raw-data/nyc-taxi/*.parquet")

# Выбираем только числовые колонки
numeric_cols = ['trip_distance', 'fare_amount', 'tip_amount', 'total_amount', 'payment_type']
df_numeric = df.select(numeric_cols)

# Конвертируем в pandas-on-Spark
psdf = df_numeric.pandas_api()
```

**Вариант 2: Извлечение компонентов даты до конвертации**
```python
from pyspark.sql import functions as F

# Извлекаем час, день недели и т.д. как числа
df_with_time = df.select(
    F.hour("tpep_pickup_datetime").alias("pickup_hour"),
    F.dayofweek("tpep_pickup_datetime").alias("pickup_dow"),
    F.month("tpep_pickup_datetime").alias("pickup_month"),
    "trip_distance", "fare_amount", "total_amount"
)

# Теперь можно конвертировать
psdf = df_with_time.pandas_api()
```

**Вариант 3: Для анализа с датами - использовать Spark DataFrame API**
```python
# Группировка по дате через Spark (работает без ограничений)
daily = df.groupBy(F.to_date("tpep_pickup_datetime").alias("date")) \
    .agg(F.count("*").alias("trips"))

# Результат агрегации уже без TimestampNTZ - можно конвертировать
daily_psdf = daily.pandas_api()
```

---

In [ ]:
import pyspark.pandas as ps

# Настройка для лучшей производительности
ps.set_option('compute.default_index_type', 'distributed')
ps.set_option('compute.ops_on_diff_frames', True)

# Вариант 1: Только числовые колонки
numeric_cols = [
    'VendorID', 'passenger_count', 'trip_distance', 'RatecodeID',
    'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount',
    'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 
    'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee'
]
df_numeric = df.select(numeric_cols)
print(f"Вариант 1: {len(numeric_cols)} числовых колонок (без timestamp)")

In [ ]:
# Вариант 2: Извлечение компонентов даты как числа
df_with_time = df.select(
    F.hour("tpep_pickup_datetime").alias("pickup_hour"),
    F.dayofweek("tpep_pickup_datetime").alias("pickup_dow"),
    F.month("tpep_pickup_datetime").alias("pickup_month"),
    F.to_date("tpep_pickup_datetime").cast("string").alias("pickup_date"),  # дата как строка
    "trip_distance", "fare_amount", "tip_amount", "total_amount", "payment_type"
)
print("Вариант 2: timestamp -> числовые компоненты (hour, dow, month) + date как string")

In [ ]:
# Конвертация в pandas-on-Spark (используем вариант 2 с временными компонентами)
psdf = df_with_time.pandas_api()

print(f"Тип: {type(psdf)}")
print(f"Колонки: {list(psdf.columns)}")
print()
psdf.head(10)

In [ ]:
# describe() - статистика по всем 40M+ записям, распределенно
psdf[['trip_distance', 'fare_amount', 'tip_amount', 'total_amount']].describe()

In [ ]:
# Теперь можно использовать временные компоненты в pandas-стиле!

# Анализ по часам (pandas groupby)
hourly_stats = psdf.groupby('pickup_hour').agg({
    'total_amount': 'mean',
    'trip_distance': 'mean',
    'tip_amount': 'mean'
}).round(2)

print("Средние показатели по часам дня:")
hourly_stats

In [ ]:
# Анализ по дню недели (1=Вс, 2=Пн, ..., 7=Сб)
dow_stats = psdf.groupby('pickup_dow').agg({
    'total_amount': ['mean', 'count'],
    'tip_amount': 'mean'
})

print("Статистика по дням недели:")
dow_stats

In [ ]:
# Анализ по дате (pickup_date как строка работает!)
daily_revenue = psdf.groupby('pickup_date')['total_amount'].sum().sort_index()

print("Выручка по дням (первые 10):")
daily_revenue.head(10)

In [ ]:
# Фильтрация и статистика - pandas синтаксис
print("describe() - статистика по 40M+ записям:")
psdf[['trip_distance', 'fare_amount', 'tip_amount', 'total_amount']].describe()

In [ ]:
# Фильтрация - pandas синтаксис, распределенное выполнение
morning_rush = psdf[(psdf['pickup_hour'] >= 7) & (psdf['pickup_hour'] <= 9)]
evening_rush = psdf[(psdf['pickup_hour'] >= 17) & (psdf['pickup_hour'] <= 19)]

print(f"Утренний час пик (7-9): {len(morning_rush):,} поездок")
print(f"Вечерний час пик (17-19): {len(evening_rush):,} поездок")
print()
print("Сравнение:")
print(f"  Утро - средняя сумма: ${morning_rush['total_amount'].mean():.2f}")
print(f"  Вечер - средняя сумма: ${evening_rush['total_amount'].mean():.2f}")

## 11. Сохранение результатов в S3

In [ ]:
# Сохранение месячной агрегации
monthly.write.mode("overwrite").parquet("s3a://warehouse/aggregated/nyc-taxi-monthly")
print("Saved: s3a://warehouse/aggregated/nyc-taxi-monthly")

In [ ]:
# Сохранение почасовой агрегации
hourly.write.mode("overwrite").parquet("s3a://warehouse/aggregated/nyc-taxi-hourly")
print("Saved: s3a://warehouse/aggregated/nyc-taxi-hourly")

In [ ]:
# Сохранение партиционированных данных (sample)
# Берем 1% данных для примера партиционирования
df_sample = df.sample(fraction=0.01) \
    .withColumn("year", F.year("tpep_pickup_datetime")) \
    .withColumn("month", F.month("tpep_pickup_datetime"))

df_sample.write \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .parquet("s3a://warehouse/processed/nyc-taxi-partitioned")

print("Saved: s3a://warehouse/processed/nyc-taxi-partitioned (partitioned by year/month)")

## 12. Работа с Spark UI

### Доступ к Spark UI

**Docker Compose:** http://localhost:4040

**Kubernetes:**
```bash
kubectl port-forward -n spark svc/spark-connect 4040:4040
# Затем открыть http://localhost:4040
```

### Вкладки Spark UI

| Вкладка | Описание |
|---------|----------|
| **Jobs** | Список всех jobs и их статус |
| **Stages** | Детали stages (tasks, input/output size, shuffle) |
| **Storage** | Кэшированные RDD/DataFrames |
| **Environment** | Конфигурация Spark |
| **Executors** | Метрики executors (memory, GC, tasks) |
| **SQL** | Query plans для SQL/DataFrame операций |

### Полезные метрики в SQL вкладке:
- **Duration** - время выполнения запроса
- **Rows output** - количество строк на каждом этапе
- **Data size** - объем данных
- **Physical Plan** - план выполнения (кликните на query)

### Spark History Server

**Docker Compose:** http://localhost:18080

**Kubernetes:**
```bash
kubectl port-forward -n spark svc/spark-history-server 18080:18080
# Затем открыть http://localhost:18080
```

После завершения приложения (`spark.stop()`) можно просмотреть историю выполненных jobs.

In [ ]:
# Просмотр плана выполнения
complex_query = df.filter(F.col("fare_amount") > 10) \
    .groupBy(F.month("tpep_pickup_datetime").alias("month")) \
    .agg(
        F.count("*").alias("trips"),
        F.avg("total_amount").alias("avg_total")
    ) \
    .orderBy("month")

# Логический план
print("=== Logical Plan ===")
complex_query.explain(mode="simple")

In [ ]:
# Расширенный план (физический)
print("=== Physical Plan ===")
complex_query.explain(mode="extended")

## 13. Tips: Spark Connect vs sparkmagic

### Основные отличия

| Аспект | sparkmagic | Spark Connect |
|--------|------------|---------------|
| Протокол | Livy REST API | gRPC |
| Latency | Высокая (HTTP) | Низкая (gRPC) |
| Сессия | Через Livy сервер | Прямое подключение |
| Magic commands | `%%spark`, `%%sql` | Нативный Python |

### Эквивалентные операции

**sparkmagic:**
```python
%%spark
df = spark.read.parquet("s3a://...")
df.show()
```

**Spark Connect:**
```python
df = spark.read.parquet("s3a://...")
df.show()
```

**sparkmagic SQL:**
```python
%%sql
SELECT * FROM table
```

**Spark Connect SQL:**
```python
spark.sql("SELECT * FROM table").show()
```

### Ограничения Spark Connect

Некоторые операции недоступны в Spark Connect:
- `df.rdd` - RDD API не поддерживается
- `spark.sparkContext` - SparkContext недоступен
- UDF через `@udf` - используйте `spark.udf.register()`
- Некоторые методы `df.pandas_api()` - используйте `pyspark.pandas` напрямую

### Автоматическое подключение (как в sparkmagic)

Для автоматического создания сессии при запуске ноутбука, см. `/home/jupyter/.ipython/profile_default/startup/00-spark.py`

In [ ]:
# Конвертация в pandas для локальной работы
# ВНИМАНИЕ: загружает данные на клиент, используйте для небольших результатов

# Безопасно - небольшой агрегат
monthly_pd = monthly.toPandas()
print(f"Тип: {type(monthly_pd)}")
monthly_pd.head()

In [ ]:
# Очистка кэша
df.unpersist()
print("Cache cleared")

In [ ]:
# Завершение сессии
# Docker Compose: после этого приложение появится в History Server (http://localhost:18080)
spark.stop()
print("Session stopped.")